In [ ]:
# test_car_color.ipynb

import torch
from torchvision import models, transforms
from PIL import Image
import matplotlib.pyplot as plt
import os

# Configurações
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = 'car_color_model.pt'

# Transformações (iguais ao treino)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# Função para carregar e prever a imagem
def predict_image(img_path, model, class_names):
    img = Image.open(img_path).convert('RGB')
    img_tensor = transform(img).unsqueeze(0).to(device)

    model.eval()
    with torch.no_grad():
        output = model(img_tensor)
        _, predicted = torch.max(output, 1)

    return class_names[predicted.item()], img

# Carregar modelo treinado
num_classes = len(os.listdir('data/train'))
class_names = sorted(os.listdir('data/train'))

model = models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)

# 📸 Teste com imagem (adicione sua imagem no mesmo diretório do notebook)
img_path = 'carro_teste.jpg'  # Substitua com o nome da imagem que você quiser testar
predicted_class, img = predict_image(img_path, model, class_names)

# Visualização
plt.imshow(img)
plt.axis('off')
plt.title(f'Predição: {predicted_class}', fontsize=16)
plt.show()
